In [1]:
%run "..\\config\\AppSettings.py"

import pandas as pd
#from app.config.AppSettings import AppSettings

In [2]:
settings = AppSettings()

url_list = []
url_list.append(settings._common_account)
url_list.append(settings._repo_name)
url_list.append(settings._branch_name)
url_list.append(settings._app_root)
url_list.append(settings._data_analysis_folder)
url_list.append(settings._cleaned_files_path)
url_list.append(settings._cleaned_csv_path)

csv_url = settings._raw_data_url + "/".join(url_list)

#Reading Cleaned CSV
df = pd.read_csv(csv_url, sep=",", header=0, skipinitialspace=True)

In [3]:
#Overall picture of the data
df.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Car_Age,Brand,Model
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998.0,58.16,5.0,0,1.75,14,Maruti,Wagon
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582.0,126.20,5.0,0,12.50,9,Hyundai,Creta
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199.0,88.70,5.0,8.61 Lakh,4.50,13,Honda,Jazz
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248.0,88.76,7.0,0,6.00,12,Maruti,Ertiga
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968.0,140.80,5.0,0,17.74,11,Audi,A4


In [4]:
#Subset of data for further processing
df_group_by = df[["Fuel_Type", "Kilometers_Driven", "Price", "Car_Age"]]

In [5]:
#Arithmetic Mean
df_group_by.groupby("Fuel_Type").mean()

,Kilometers_Driven,Price,Car_Age
Fuel_Type,,,
CNG,53228.935484,3.176452,9.951613
Diesel,67550.011163,10.683837,10.241173
Electric,47000.000000,12.875000,10.500000
LPG,60624.250000,2.072500,13.416667
Petrol,48547.361504,4.708337,11.093233


In [6]:
#Median
df_group_by.groupby("Fuel_Type").median()

,Kilometers_Driven,Price,Car_Age
Fuel_Type,,,
CNG,53000.0,3.150,10.0
Diesel,60000.0,6.460,10.0
Electric,47000.0,12.875,10.5
LPG,64636.5,2.310,13.5
Petrol,45000.0,3.490,10.0


In [7]:
#Minimum
df_group_by.groupby("Fuel_Type").min()

,Kilometers_Driven,Price,Car_Age
Fuel_Type,,,
CNG,10600,0.00,5
Diesel,1000,0.00,5
Electric,44000,12.75,8
LPG,17000,0.00,10
Petrol,171,0.00,5


In [8]:
#Maximum
df_group_by.groupby("Fuel_Type").max()

,Kilometers_Driven,Price,Car_Age
Fuel_Type,,,
CNG,100000,8.35,19
Diesel,6500000,160.00,28
Electric,50000,13.00,13
LPG,87829,3.20,17
Petrol,620000,120.00,26


In [9]:
#Sum
df_group_by.groupby("Fuel_Type").sum()

,Kilometers_Driven,Price,Car_Age
Fuel_Type,,,
CNG,3300194,196.94,617
Diesel,260202643,41154.14,39449
Electric,94000,25.75,21
LPG,727491,24.87,161
Petrol,161419977,15655.22,36885


In [10]:
#Creating data frame for pivot table processing
df_pivot = df[["Location", "Year", "Fuel_Type", "Kilometers_Driven", "Price", "Car_Age"]]

In [11]:
# Pandas Pivot table with defaults (Aggregation is mean)
pd.pivot_table(df_pivot, index=["Location", "Year", "Fuel_Type"])

Car_Age  Kilometers_Driven      Price
Location  Year Fuel_Type                                       
Ahmedabad 2002 Petrol        22.0       85000.000000   1.200000
          2004 Petrol        20.0       60000.000000   1.400000
          2005 Petrol        19.0       69500.000000   1.130000
          2007 Petrol        17.0       62326.571429   1.735714
          2008 Diesel        16.0       56251.750000   5.812500
...                           ...                ...        ...
Pune      2017 Diesel         7.0       53659.482759  13.765172
               Petrol         7.0       20710.473684   6.156842
          2018 Diesel         6.0       21758.181818  13.527273
               Petrol         6.0       15065.416667  10.408333
          2019 Petrol         5.0        1007.500000   4.300000

[400 rows x 3 columns]

In [12]:
# Pivot table with specific subset of columns (Aggregation is Sum)
pd.pivot_table(df_pivot, index=["Location", "Year", "Fuel_Type"],
               values=[ "Kilometers_Driven"], aggfunc="sum")

Kilometers_Driven
Location  Year Fuel_Type                   
Ahmedabad 2002 Petrol                 85000
          2004 Petrol                 60000
          2005 Petrol                139000
          2007 Petrol                436286
          2008 Diesel                225007
...                                     ...
Pune      2017 Diesel               1556125
               Petrol                393499
          2018 Diesel                239340
               Petrol                180785
          2019 Petrol                  2015

[400 rows x 1 columns]

In [13]:
# Pivot table with multiple Aggregations
pd.pivot_table(df_pivot, index=["Location", "Year", "Fuel_Type"], 
               values=["Kilometers_Driven", "Price", "Car_Age"], aggfunc=["sum", "count"], margins=True)

sum                               count  \
                         Car_Age Kilometers_Driven     Price Car_Age   
Location  Year Fuel_Type                                               
Ahmedabad 2002 Petrol         22             85000      1.20       1   
          2004 Petrol         20             60000      1.40       1   
          2005 Petrol         38            139000      2.26       2   
          2007 Petrol        119            436286     12.15       7   
          2008 Diesel         64            225007     23.25       4   
...                          ...               ...       ...     ...   
Pune      2017 Petrol        133            393499    116.98      19   
          2018 Diesel         66            239340    148.80      11   
               Petrol         72            180785    124.90      12   
          2019 Petrol         10              2015      8.60       2   
All                        77133         425744305  57056.92    7253   

                                                  
                         Kilometers_Driven Price  
Location  Year Fuel_Type                          
Ahmedabad 2002 Petrol                    1     1  
          2004 Petrol                    1     1  
          2005 Petrol                    2     2  
          2007 Petrol                    7     7  
          2008 Diesel                    4     4  
...                                    ...   ...  
Pune      2017 Petrol                   19    19  
          2018 Diesel                   11    11  
               Petrol                   12    12  
          2019 Petrol                    2     2  
All                                   7253  7253  

[401 rows x 6 columns]